In [38]:
import pymssql
import pandas as pd

In [39]:
def dataframe_from_query(given_cursor, given_query):
    given_cursor.execute(given_query)
    return_data = given_cursor.fetchall()
    dfr_columns = [item[0] for item in given_cursor.description]
    return_dataframe = pd.DataFrame(data=return_data, columns=dfr_columns)
    
    return return_dataframe

In [40]:
conn = pymssql.connect(
    server='10.100.200.3',
    port='1433',
    user='pawel', 
    password='20Al3Mot@VP@weL22#',
    database='SPEED',
    charset='ISO-8859-2')
cursor = conn.cursor()

### Czy są niezafakturowane zlecenia, pod którymi są koszty?

In [41]:
zlecenia_query = """
    SELECT * FROM [SPEED].[dbo].[ZLECENIA]  WHERE LOKALIZACJA = 'KAT_MS' AND YEAR([ZA_DATA]) = 2021"""

In [42]:
zlecenia_df = dataframe_from_query(cursor, zlecenia_query).fillna('')

In [43]:
faktura_cols = [col for col in zlecenia_df.columns if 'FAKT' in col]

In [44]:
zlecenia_df.loc[zlecenia_df.ZAFAKTUROWANE != 1].loc[zlecenia_df.FAKTURA_K != ''][['NR_ZLECENIA'] + faktura_cols][:3]

,NR_ZLECENIA,FAKTURA,FAKTURA_DATA,FAKTURA_ZB_ID,FAKTURA_ZB,FAKTURA_PF,FAKTURA_K,FAKTURA_K_ZB_ID,FAKTURA_K_ZB,ZAFAKTUROWANE
301,00119/S/02/2021/JG/MS,,,,,,FA/3918/02/2021/TVM,,,0
576,00125/S/03/2021/AB/MS,,,,,,215/03/2021,,,0
643,00192/S/03/2021/JG/MS,,,,,,FA/6388/03/2021/TVM,,,0


Istnieją niezafakturowane zlecenia, pod którymi są koszty

### Czy są faktury bez zleceń?

In [45]:
faktury_query = """
    SELECT * FROM [SPEED].[dbo].[FAKTURY] WHERE YEAR([DATA_WYSTAWIENIA]) = 2021 
"""

In [46]:
faktury_df = dataframe_from_query(cursor, faktury_query).fillna('')

In [48]:
zlecenia_cols = [col for col in faktury_df.columns if 'ZLEC' in col]

In [54]:
faktury_df.loc[faktury_df.ZLECENIE == ''][zlecenia_cols].describe

<bound method NDFrame.describe of       ZLECENIE_ID ZLECENIE  ZLECENIE_C_ID ZLECENIE_C  ZLECENIE_M_ID  \
19              0                       0                         0   
24              0                       0                         0   
27              0                       0                         0   
32              0                       0                         0   
36              0                       0                         0   
...           ...      ...            ...        ...            ...   
9460            0                       0                         0   
9461            0                       0                         0   
9507            0                       0                         0   
9618            0                       0                         0   
9660            0                       0                         0   

     ZLECENIE_M  ZLECENIE_RAIL_ID ZLECENIE_RAIL  ZLECENIE_ZB_ID ZLECENIE_ZB  \
19                              0 